In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
#%matplotlib inline
import seaborn as sns

import pylab as pl
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import cluster
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_curve
from collections import Counter
import scipy.stats as stats
from scipy.stats import chisquare

import mpld3

from sklearn.ensemble import RandomForestClassifier

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Cow data

In [3]:
file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/df_sensor_cow_completedates_mergedclinicaldata_6H12Haggreg_August2020.csv"
df = pd.read_csv(file_path, index_col = False)


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,36,53,58,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<br>
In the file with all the cow data, there are the following variables:
> date_time_stamp [2020-06-15 16:00:00]: this is the hourly data in UTC time zone<br>
> date_time_stamp_duplicate [2020-06-15 16:00:00-07:00]: this is the hourly data transformed into PCT time zone<br>
> TimeStamp [20200615 16:00:00]: this is the hourly data, in UTC, that came with the sensor file<br>
<br>
> cow_id [172]: an int64

### Sensor data - mean values

In [4]:
file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_rumination_diff_6Hmean.csv"
df_rumination_6Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_eating_diff_6Hmean.csv"
df_eating_6Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_notactive_diff_6Hmean.csv"
df_notactive_6Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_active_diff_6Hmean.csv"
df_active_6Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_highactivity_diff_6Hmean.csv"
df_highactivity_6Hmean = pd.read_csv(file_path, index_col = False)



file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_rumination_diff_12Hmean.csv"
df_rumination_12Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_eating_diff_12Hmean.csv"
df_eating_12Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_notactive_diff_12Hmean.csv"
df_notactive_12Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_active_diff_12Hmean.csv"
df_active_12Hmean = pd.read_csv(file_path, index_col = False)

file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/dataframes_behaviors_6H12H_meanvarsum/df_highactivity_diff_12Hmean.csv"
df_highactivity_12Hmean = pd.read_csv(file_path, index_col = False)

<br>
In the file with sensor data, there are the following variables:
> cow_id [413]: <br>
> datetime_stamp [2020-06-30 13:00:00]<br>
> outcome variable (scc_catg [0.0, 1.0]) <br>
> raw sensor data at time 0 (rumination_diff_6Hmean) <br>
> t_minus1, t_minus2, etc: they have the word 'missing' so I could keep track of missing values<br>

In [5]:
for i in df_rumination_6Hmean:
    print (df_rumination_6Hmean[i].value_counts())

7166    1
7494    1
5707    1
7244    1
5453    1
6989    1
7249    1
4690    1
1738    1
7584    1
6488    1
3417    1
1627    1
1884    1
4958    1
3682    1
7553    1
1896    1
1897    1
7274    1
626     1
1654    1
7032    1
5497    1
7546    1
7037    1
9855    1
6273    1
3716    1
2889    1
       ..
2344    1
5831    1
400     1
7057    1
7059    1
3988    1
5781    1
7430    1
5785    1
7439    1
7678    1
413     1
7328    1
674     1
6819    1
6821    1
4518    1
8872    1
172     1
2478    1
7282    1
7089    1
5555    1
1230    1
7048    1
3006    1
6849    1
7107    1
7463    1
3585    1
Name: cow_id_x, Length: 123, dtype: int64
2020-06-30 20:00:00    45
2020-08-04 20:00:00    43
2020-06-02 20:00:00    35
Name: datetime_stamp, dtype: int64
0.0    89
1.0    34
Name: scc_catg, dtype: int64
5521.0    1
7138.0    1
7546.0    1
5707.0    1
5497.0    1
6895.0    1
7373.0    1
9425.0    1
7140.0    1
5386.0    1
7164.0    1
7059.0    1
7188.0    1
7222.0    1
7244.0    1
3682.0

Name: t_minus33, Length: 71, dtype: int64
1.5                    5
-2.1666666666666665    5
-0.6666666666666666    5
1.1666666666666667     4
0.8333333333333334     4
-1.1666666666666667    3
-2.3333333333333335    3
3.3333333333333335     3
-4.666666666666667     3
-4.0                   3
-0.8333333333333334    3
1.8333333333333333     3
5.166666666666667      2
0.16666666666666666    2
4.166666666666667      2
4.666666666666667      2
missing                2
2.6666666666666665     2
4.5                    2
1.6666666666666667     2
6.0                    2
6.666666666666667      2
3.0                    2
-2.8333333333333335    2
-6.833333333333333     2
1.0                    2
-0.3333333333333333    2
4.833333333333333      2
-1.8333333333333333    2
7.5                    2
                      ..
0.0                    1
8.333333333333334      1
2.0                    1
-3.6666666666666665    1
13.0                   1
5.5                    1
-2.5                   1
3.166666

Dataframe with cow data:<br>
Outcomes:<br>
- scc_catg
- hygiene_score
<br><br>
Milk variables:<br>
- Fat
- Lac
- Mun
- Pro
- Snf
- Last test-day milk production (lbs)
<br><br>
Cow variables:<br>
- cow_id
- hygiene_score
- parity_catg
- lactation number
- DIM
- diff_dryoff_lactCM
- No_CM_catg (0 for no cases, 1 for at least 1 case of mastitis)
- N clinical mastitis current lactation (total number of mastitis cases)
- N days for 1st CM case (use only if we want to study a subset of animals that had at least 1 CM; too many NaN otherwise)
- N days from last CM to dry-off (use only if we want to study a subset of animals that had at leas 1 CM; too many NaN otherwise)
<br><br>
Environmental variables:
- season
- month
- 'Pen' vs. 'last Pen'

Dataframe with sensor data:<br>
Cow variables:<br>
- cow_id
<br><br>
Sensor variables:<br>
- t_minus1, t_minus2, t_minus3, etc.<br><br>

## 1. Create dataframes with features and labels
<br><br>

In [149]:
def dataframe_features_sensorONLY(dataframe_name, label_var, time_back_number, time_aggregation_number, features_sensor_listX):
    
    ''' This function is to create a dataframe with all the features I will be needing later on
    Inputs:
    - dataframe that contains the label_variable or outcome, the sensor features in the list provided,
       and the animal features provided in the list.
    - the label variable, as string.
    - features_sensor_list: the list of t_minusX features from the sensor
    - time_back_number:
    
    Output:
    It combines all the dataframes with the features into one single dataframe, and 
    a separate dataframe with the labels. It keeps the relationship with the index'''
    
    #1. Make sure all the missing values are np.nan and remove any rows with no outcome
    df = dataframe_name.replace('missing', np.nan)
    df = df.replace('NaN', np.nan)
    df = df.loc[(df[label_var].notnull())]
    
    # Select number of features based on number of days back and time aggregation
    time_back = time_back_number
    time_aggregation = time_aggregation_number
    n_sensorfeatures = (24 / time_aggregation) * time_back
    values = range(0, n_sensorfeatures)
    features_sensor_list = features_sensor_listX[0:len(values)]
    
    
    #2. Change into a number so I can standardize the sensor values
    for i in features_sensor_list:
        df[i] = df[i].astype('float')
    
    
    #3. Standardization of the sensor data, ignoring NaN. It will create a separate dataframe
    df_std = pd.DataFrame()
    for i in features_sensor_list:
        df_std_i = df[[i]].apply(lambda x:(x - np.nanmean(x)) / (np.nanstd(x)))
        df_std = pd.concat([df_std, df_std_i], axis = 1)
    
    
    #4. Concatenation of all the dataframes, sensor, and other features
    # create a list with the column names I will be using
    columns = list(df_std)
    
    # create a dataframe where the index is same as the one from the standardized sensor values
    # it needs some column to start with, so I create the 'fake' column
    # I will use it to append new columns
    df_sensorfeatures = pd.DataFrame(data = df_std.index)
    df_sensorfeatures['fake'] = 1
    df_sensorfeatures = df_sensorfeatures.set_index(0)
    
    
    for i in columns:
        df_i = df_std.loc[(df_std[i].notnull())][[i]]
        df_sensorfeatures = df_sensorfeatures.join(df_i, how = 'inner')
    
    # and drop the first column
    df_sensorfeatures = df_sensorfeatures.drop(columns = 'fake')
    
    
    #5. Selecting the label: take the column with the label and match the indexes to a list of indexes
    df_label = dataframe_name.loc[(dataframe_name[label_var].notnull())][[label_var]]
    
    list_index = df_sensorfeatures.index.values.tolist()
    df_label = df_label.loc[list_index]
    
    return (df_sensorfeatures, df_label)

In [150]:
features_sensor_list24 = ['t_minus3', 't_minus2', 't_minus1']

features_sensor_list12 = ['t_minus6', 't_minus5', 't_minus4', 't_minus3', 't_minus2', 't_minus1']

features_sensor_list6 = ['t_minus1', 't_minus2', 't_minus3', 't_minus4', 't_minus5',
                         't_minus6', 't_minus7', 't_minus8', 't_minus9', 't_minus10', 
                         't_minus11', 't_minus12', 't_minus13', 't_minus14', 't_minus15',
                         't_minus16', 't_minus17', 't_minus18', 't_minus19', 't_minus20',
                         't_minus21', 't_minus22', 't_minus23', 't_minus24', 't_minus25',
                         't_minus26', 't_minus27', 't_minus28', 't_minus29', 't_minus30',
                         't_minus31', 't_minus32', 't_minus33', 't_minus34', 't_minus35',
                         't_minus36', 't_minus37', 't_minus38', 't_minus39', 't_minus40', 
                         't_minus41', 't_minus42', 't_minus43', 't_minus44', 't_minus45',
                         't_minus46', 't_minus47', 't_minus48', 't_minus49', 't_minus50',
                         't_minus51', 't_minus52', 't_minus53', 't_minus54', 't_minus55',
                         't_minus56']

features_sensor_list3 = ['t_minus24', 't_minus23', 't_minus22', 't_minus21', 't_minus20',
                         't_minus19', 't_minus18', 't_minus17', 't_minus16', 't_minus15',
                         't_minus14', 't_minus13', 't_minus12', 't_minus11', 't_minus10',
                         't_minus9', 't_minus8', 't_minus7', 't_minus6', 't_minus5',
                         't_minus4', 't_minus3', 't_minus2', 't_minus1']

features_animal = ['cow_id_x']

In [155]:
df_features, df_label = dataframe_features_sensorONLY(df_rumination_6Hmean, 
                                                      label_var = 'scc_catg', 
                                                      time_back_number = 3, 
                                                      time_aggregation_number = 6, 
                                                      features_sensor_listX = features_sensor_list6)   

<br>
## 2. Random Search
<br><br>

In [157]:
def RandomForest_sensorvalues_RandomizedSearchCV(features_df, label_df, randomsearch_grid):
    
    # Fit Random Forest with Randomized Search CV
    # first, create the base model to tune
    rand_for = RandomForestClassifier(random_state = 42)
    
    # Random search of parameters, using 3-fold cross validation, search across 100 different combinations, all cores
    rand_for_random = RandomizedSearchCV(estimator = rand_for, 
                                         param_distributions = randomsearch_grid, 
                                         n_iter = 100, 
                                         cv = 3, 
                                         verbose = 2, 
                                         random_state = 42, 
                                         n_jobs = -1)
    # fit random search model
    rand_for_random.fit(features_df, label_df)
    x = rand_for_random.best_params_
    
    
    return x

In [158]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
my_randomsearch_grid = {'n_estimators': n_estimators,
                        'max_features': max_features,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'bootstrap': bootstrap}

In [159]:
my_randomsearch_grid

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

In [160]:
RandomForest_sensorvalues_RandomizedSearchCV(features_df = df_features, 
                                             label_df = df_label, 
                                             randomsearch_grid = my_randomsearch_grid)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=30, total=   1.9s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=30, total=   1.9s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10 


[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=900, min_samples_split=5, max_features=auto, max_depth=10, total=  12.3s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=10 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min


[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=90, total=   5.4s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=90, total=   5.5s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=90, total=   5.4s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=10, max_features=sqrt, max_depth=10, total=  14.2s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=10 
[CV]  bootstrap=False,

[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=70, total=   8.8s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=900, min_samples_split=2, max_features=auto, max_depth=None 
[CV]  bootstrap=True, min_samples_leaf=4, n_estimators=700, min_samples_split=5, max_features=sqrt, max_depth=80, total=   7.6s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, min_samples_split=5, max_features=sqrt, max_depth=80 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=900, min_samples_split=2, max_features=auto, max_depth=None, total=   9.7s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, min_samples_split=5, max_features=sqrt, max_depth=80 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=900, min_samples_split=2, max_features=auto, max_depth=None, total=   9.8s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, min_samples_split=5, max_features=sqrt, max_depth=80 
[CV]  bootstrap=True, min_

...

[CV]  bootstrap=False, min_samples_leaf=2, n_estimators=200, min_samples_split=10, max_features=sqrt, max_depth=90, total=   2.8s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=200, min_samples_split=10, max_features=sqrt, max_depth=90 
[CV]  bootstrap=False, min_samples_leaf=2, n_estimators=200, min_samples_split=10, max_features=sqrt, max_depth=90, total=   2.5s
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=200, min_samples_split=10, max_features=sqrt, max_depth=90 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=auto, max_depth=90, total=   9.6s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=800, min_samples_split=10, max_features=sqrt, max_depth=None 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=auto, max_depth=90, total=   9.6s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=800, min_samples_split=10, max_features=sqrt, max_depth=None 
[CV]  bootstrap=False,

[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=300, min_samples_split=10, max_features=sqrt, max_depth=40, total=   3.5s
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=100 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=70, total=   4.7s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=2, max_features=sqrt, max_depth=80 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=100, total=   2.3s
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=200, min_samples_split=5, max_features=sqrt, max_depth=100, total=   2.2s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=2, max_features=sqrt, max_depth=80 
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=2, max_features=sqrt, max_depth=80 
[CV]  bootstrap=True, min_samp

[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=100, min_samples_split=5, max_features=auto, max_depth=100, total=   1.1s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=2, max_features=sqrt, max_depth=20 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=500, min_samples_split=2, max_features=auto, max_depth=10, total=   5.6s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=2, max_features=sqrt, max_depth=20 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=500, min_samples_split=2, max_features=auto, max_depth=10, total=   5.7s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=2, max_features=sqrt, max_depth=20 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=500, min_samples_split=2, max_features=auto, max_depth=10, total=   5.5s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=10, max_features=auto, max_depth=40 
[CV]  bootstrap=True, min_

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.4min finished
/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 800}

<br>
## 3. Fit Random Forest using results from random search
<br><br>

In [98]:
def RandomForest_sensorvalues(features_df, label_df, bootstrap_bool, depth, features, samples_leaf, samples_split, estimators_no):
    
    #1. Fit the model
    rand_for = RandomForestClassifier(bootstrap = bootstrap_bool,
                                      max_depth = depth,
                                      max_features = features,
                                      min_samples_leaf = samples_leaf,
                                      min_samples_split = samples_split,
                                      n_estimators = estimators_no, random_state = 42, oob_score = True)
    
    model = rand_for.fit(features_df, label_df)
    oob_err = 1 - (rand_for.oob_score)
    pred = rand_for.predict(features_df)
    accuracy = metrics.accuracy_score(label_df, pred)
    
    conf_matrix = metrics.confusion_matrix(label_df, pred)
    classif_report = metrics.classification_report(label_df, pred)
    
    feature_imp = pd.Series(rand_for.feature_importances_, index = list(features_df)).sort_values(ascending = False)
    
    #2. Cross-Validation
    cv_scores = cross_val_score(rand_for, features_df, label_df, cv = 5, scoring = 'neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    cv_mean = rmse_scores.mean()
    cv_std = rmse_scores.std()
    
    cv_labels_pred = cross_val_predict(rand_for, features_df, label_df, cv = 5)
    cv_conf_matrix = metrics.confusion_matrix(label_df, cv_labels_pred)
    cv_class_report = classification_report(label_df, cv_labels_pred)
    
    #3. Precision-Recall Curve
    cv_y_scores = cross_val_predict(rand_for, features_df, label_df, cv = 5, method = 'predict_proba')
    precision, recall, thresholds = metrics.precision_recall_curve(label_df, cv_y_scores[:,1]) # score = prob of positive class
    auc_ROC = metrics.roc_auc_score(label_df, cv_y_scores[:, 1])
    auc_PRcurve = metrics.auc(label_df, cv_y_scores[:, 1], reorder = True)
    avg_prec = metrics.average_precision_score(label_df, cv_y_scores[:, 1])
     
    return (pred, model,
            {'oob_err': oob_err,
             'accuracy': accuracy},
            {'feature_imp': feature_imp},
            {'rmse_scores':rmse_scores,
             'cv_mean':cv_mean,
             'cv_std':cv_std},
            {'cv_conf_matrix':cv_conf_matrix},
            {'cv_class_report':cv_class_report},
            {'auc_ROC' : auc_ROC,
             'auc_PRcurve': auc_PRcurve,
             'avg precision' : avg_prec})

In [161]:
RandomForest_sensorvalues(df_features, df_label, 
                          bootstrap_bool = True, 
                          depth = 10, 
                          features = 'sqrt', 
                          samples_leaf = 4, 
                          samples_split = 2, 
                          estimators_no = 800)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]),
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=10, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=4, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
             oob_score=True, random_state=42, verbose=0, warm_start=False),
 {'accuracy': 0.8728813559322034, 'oob_err': 0},
 {'featur

<br>
## 4. Store results from model into dataframe
<br><br>